
#  Building a minC (minimum C) compiler


Enter your name and student ID.

 * Name:
 * Student ID:



<a name="intro"> </a>
# 1. Introduction
* This is the instruction for those who choose Option A for the term report
* You build a compiler for a minimum subset of C language, dubbed 'minC'
* This page explains how you should work on it, but <font color="red">your actual work is supposed to be done mostly in terminal and text editor</font>



# 2. Files
* `parser/` : a C -> XML parser written in Python + tatsu (parser generator), which converts a C source into an equivalent XML
* `{ml,jl,go,rs}/minc` : a directory for each language
* `test/` : test programs

# 3. Parser (C -> XML)
* `minc_grammar.y` : grammar definition of the minimum C language we work on, which is converted into a working python program (`minc_grammar.py`) using [tatsu](https://github.com/neogeny/TatSu) parser generator library

In [ ]:
cd ~/notebooks/pl09_minc/parser
tatsu minc_grammar.y > minc_grammar.py

* `minc_to_xml.py` : the main parser program that drives minc_grammar.py to convert a C source into an equivalent XML

In [ ]:
python3 minc_to_xml.py example/ex.c > ex.xml

* examine the result and understand how each C construct is converted into XML

In [ ]:
cat example/ex.c 

In [ ]:
cat ex.xml

* you don't have to modify `minc_grammar.y` or `minc_to_xml.py` unless you extend the grammar for extra points
* yet you are encouraged to see how simple does tatsu (or any parser generator, for that matter) make it to write a parser; just take a look at `minc_grammar.y`

##  Note: why are we using XML?
* it is unnecessary and unusual to convert the source program first into XML and then to the abstract syntax tree
* more commonly, you use a parser generator for the language you write your compiler with (OCaml, Julia, Go, or Rust), which allows you to directly build the abstract syntax tree you can manipulate in that language
* for example, C/C++ has [flex](https://en.wikipedia.org/wiki/Flex)/[bison](https://en.wikipedia.org/wiki/GNU_Bison) parser generator, whose grammar description file (analogous to minc_grammar.y) allows you to build any C/C++ data structure in it; OCaml has [ocamllex/ocamlyacc](https://v2.ocaml.org/manual/lexyacc.html) ([Menhir](http://gallium.inria.fr/~fpottier/menhir/) is a newer version of ocamlyacc).  there is a parser generator that supports multiple languages, most notably [ANTLR](https://www.antlr.org/), which supports Java and Python code generation.  in circumstances where there is a tool available for your language, it is much more straightforward and convenient to use these tools without going through XML
* the reasons why we go through XML are
  * I could not find a popular parser generator for some of the languages (Go and Julia)
  * even if one exists in each language, there will be differences between them that make it difficult/tricky/cumbersome to explain them
* so I arrived at a parser generator (tatsu) for Python as a common middle ground and XML as a common data structure all languages can easily read


# 4. {go,jl,ml,rs}/minc
* in each language-specific directory (`go, jl, ml, rs`) , there is a toplevel directory `minc`
* the code given there is a skeleton of a compiler that reads an XML file, builds its abstract syntax tree, and finally calls the code generator
* the code generator is currently almost empty and raises an exception when called
* your main job in the assignment is to complete the code generator

## 4-1. Go
###  files
* `go/`
  * `minc/`
    * `minc_ast.go` --- abstract syntax tree (AST) definition
    * `minc_parse.go` --- XML -> AST converter
    * `minc_cogen.go` --- AST -> assembly code
    * `minc.go` --- the main file

###  build

In [ ]:
export PATH=~/go/bin:$PATH
cd ~/notebooks/pl09_minc/go/minc
go build

###  run
* be sure you have generated ex.xml by `minc_to_xml.py`
* try to compile a small program and see that the code generator raises an exception


In [ ]:
./minc ../../parser/ex.xml ex.s

* take a look at the source code that caused the exception

In [ ]:
cat minc_cogen.go

* your job is to implement `ast_to_asm_program` function

## 4-2. Julia
###  files
* `jl/`
  * `minc/`
    * `minc_ast.jl` --- abstract syntax tree (AST) definition
    * `minc_parse.jl` --- XML -> AST converter
    * `minc_cogen.jl` --- AST -> assembly code
    * `minc.jl` --- the main file

###  build

In [ ]:
export PATH=~/.juliaup/bin:$PATH
cd ~/notebooks/pl09_minc/jl/minc
chmod +x minc.jl

###  run
* be sure you have generated ex.xml by `minc_to_xml.py`
* try to compile a small program and see that the code generator raises an exception


In [ ]:
./minc.jl ../../parser/ex.xml ex.s

* take a look at the source code that caused the exception

In [ ]:
cat minc_cogen.jl

* your job is to implement `ast_to_asm_program` function

## 4-3. OCaml
###  files
* `ml/`
  * `minc/`
    * `libs/`
      * `minc_ast.ml` --- abstract syntax tree (AST) definition
      * `minc_parse.ml` --- XML -> AST converter
      * `minc_cogen.ml` --- AST -> assembly code
      * `dune` --- describes dependencies between them
    * `bin/`
      * `main.ml` --- the main file

###  build

In [ ]:
eval $(opam env)
cd ~/notebooks/pl09_minc/ml/minc
dune build

###  run
* be sure you have generated ex.xml by `minc_to_xml.py`
* try to compile a small program and see that the code generator raises an exception


In [ ]:
_build/default/bin/main.exe ../../parser/ex.xml ex.s

* take a look at the source code that caused the exception

In [ ]:
cat lib/minc_cogen.ml

* your job is to implement `ast_to_asm_program` function

## 4-4. Rust
###  files
* `rs/`
  * `minc/`
    * `src/`
      * `minc_ast.rs` --- abstract syntax tree (AST) definition
      * `minc_parse.rs` --- XML -> AST converter
      * `minc_cogen.rs` --- AST -> assembly code
      * `main.rs` --- the main file

###  build

In [ ]:
. ~/.cargo/env
cd ~/notebooks/pl09_minc/rs/minc
cargo build

###  run
* be sure you have generated ex.xml by `minc_to_xml.py`
* try to compile a small program and see that the code generator raises an exception


In [ ]:
./target/debug/minc ../../parser/ex.xml ex.s

* take a look at the source code that caused the exception

In [ ]:
cat src/minc_cogen.rs

* your job is to implement `ast_to_asm_program` function


# 5. test
* your primary goal is to pass all the tests

##  files
* `test/`
  * `src/` 
    * `f00.c, f01.c, f02.c, ..., ` --- test programs each definint a function `f`
  * `main.c` --- a file that calls function `f`
  * `Makefile` --- executes all the tests

the `Makefile` does the following for each test program (`src/f00.c, src/f01.c,` ...)

1. convert `src/fXX.c` to `xml/fXX.xml` with `parser/minc_to_xml.py`
1. compile `xml/fXX.xml` to `minc/fXX.s` with the minC compiler you are supposed to write
1. compile `main.c` and `minc/fXX.s` into an executable
1. compile `main.c` and `src/fXX.c` into an executable with gcc
1. execute the two executables and compare their outputs

* in the `Makefile`
 * <font color=red>you must set the variable `mincc` to the path to your compiler (relative to the test directory)</font>
```
mincc := ../ml/minc/_build/default/bin/main.exe
```
 * you can set the variable `srcs` to change the functions tested. the default is all the 75 files in `src/`
```
srcs := $(wildcard src/f*.c)
```
for example, if you set this to
```
srcs := $(wildcard src/f00.c)
```
you can only test `src/f00.c`

 * `make -n` will show you what will be executed without actually executing it
 

In [5]:
# make sure you set minc variable in Makefile
cd ~/notebooks/pl09_minc/test
make -n minc=../jl/minc/minc.jl

mkdir -p xml/dir
echo "# convert src/f00.c to xml/f00.xml"
python3 ../parser/minc_to_xml.py src/f00.c > xml/f00.xml
mkdir -p asm/dir
echo "# compile xml/f00.xml to asm with your minC compiler"
../jl/minc/minc.jl xml/f00.xml asm/f00.s
mkdir -p minc/dir
echo "# generate the executable that calls f with your minC compiler"
gcc -o minc/f00.exe -DTEST_NO=0 main.c asm/f00.s -O0 -g
mkdir -p out/dir
echo "# run the executable generated by gcc"
minc/f00.exe | tee out/f00.minc
mkdir -p gcc/dir
echo "# generate the executable that calls f with gcc"
gcc -o gcc/f00.exe -DTEST_NO=0 main.c src/f00.c -O0 -g
echo "# run the executable generated by gcc"
gcc/f00.exe | tee out/f00.gcc
echo "# take the diff of the two"
diff out/f00.gcc out/f00.minc > out/f00.diff
echo "# convert src/f01.c to xml/f01.xml"
python3 ../parser/minc_to_xml.py src/f01.c > xml/f01.xml
echo "# compile xml/f01.xml to asm with your minC compiler"
../jl/minc/minc.jl xml/f01.xml asm/f01.s
echo "# generate the executable that calls f wi

to test a single file

In [6]:
make -n srcs=src/f00.c minc=../jl/minc/minc.jl

mkdir -p xml/dir
echo "# convert src/f00.c to xml/f00.xml"
python3 ../parser/minc_to_xml.py src/f00.c > xml/f00.xml
mkdir -p asm/dir
echo "# compile xml/f00.xml to asm with your minC compiler"
../jl/minc/minc.jl xml/f00.xml asm/f00.s
mkdir -p minc/dir
echo "# generate the executable that calls f with your minC compiler"
gcc -o minc/f00.exe -DTEST_NO=0 main.c asm/f00.s -O0 -g
mkdir -p out/dir
echo "# run the executable generated by gcc"
minc/f00.exe | tee out/f00.minc
mkdir -p gcc/dir
echo "# generate the executable that calls f with gcc"
gcc -o gcc/f00.exe -DTEST_NO=0 main.c src/f00.c -O0 -g
echo "# run the executable generated by gcc"
gcc/f00.exe | tee out/f00.gcc
echo "# take the diff of the two"
diff out/f00.gcc out/f00.minc > out/f00.diff



* if the test fails, identify which file it failed for and how

* use
```
make srcs=src/fXX.c
```
to run the test the file that failed

* it first converts the C source file into XML using minc_to_xml.py; if the test fails here, it's not your fault, unless you modified the grammar

```
echo "# convert src/f00.c to xml/f00.xml"
../parser/minc_to_xml.py src/f00.c > xml/f00.xml
```

* next it calls your compiler to convert the XML file into asm (the command line depends on the language you chose)

```
echo "# compile xml/f00.xml to asm with your minC compiler"
../ml/minc/_build/default/bin/main.exe xml/f00.xml asm/f00.s
```

if this fails, examine the original C source file (f00.c in the case above) and XML source file to examine what kind of source code makes it fail

* then it calls the gcc to compile the assembly you generated into an executable

```
echo "# generate the executable that calls f with your minC compiler"
gcc -o minc/f00.exe -DTEST_NO=0 main.c asm/f00.s -O0 -g
```

if you generate a syntactically invalid assembly code, gcc will complain here.  read the gcc error message, examine the assembly you generated (asm/f00.s in the case above) and understand why it failed

* then it calls executables generated by gcc as well as your minC compiler

```
echo "# run the executable generated by gcc"
./gcc/f00.exe | tee out/f00.gcc
echo "# run the executable generated by your minC compiler"
./minc/f00.exe | tee out/f00.minc
```
the gcc-generated executable is unlikely to fail.  if your compiler-generated executable fails, you might be able to see the reason just by looking at the assembly code you generated; otherwise, run the executable with a debugger. GDB, for example, can step-execute an assembly program and allow you to examine the value of registers.

* it finally compares the output of the two executables

```
echo "# take the diff of the two"
diff out/f00.gcc out/f00.minc
```

if it fails, the debugging strategy is the same as above; you might be able to see the reason just by looking at the assembly code you generated; otherwise, run the executable with a debugger. GDB, for example, can step-execute an assembly program and allow you to examine the value of registers.

## 5-1. How to add new test programs
* if you work on any of the extension, you are likely to add test programs too
* src/src/fun.c contains all the C functions to test, each one being guarded by
```
#if TEST_NO == ???

#endif
```
add your test case at the end of the file, using the next number as your TEST_NO
* modify the following line in `test/src/src/Makefile`
```
tests := $(shell seq -f "%02.0f" 0 74)
```
to reflect the tests you added.  for example, if you add two test cases (TEST_NO = 75 and 76), you should change it to 
```
tests := $(shell seq -f "%02.0f" 0 76)
```
and run make in `test/src/src`

In [ ]:
cd ~/notebooks/pl09_minc/test/src/src
make

* do a similar thing for `test/main.c`; as long as all functions take only `long` arguments and return a `long` value, you can use the same `main` function. in that case you just change
```
#if 0 <= TEST_NO && TEST_NO <= 74
...
#endif
```
to, say, 
```
#if 0 <= TEST_NO && TEST_NO <= 76
...
#endif
```

* if you support types other than `long`, you are likely to add a different main function. modify `main.c` accordingly



# 6. Format of the report and how to submit your work
## 6-1. Baseline requirements (Level 1)
* implement the compiler for minC (you will mainly write `minc_cogen.{ml,jl,go,rs}`)
* your code generator must be heavily commented (explain how it works, as if you are writing a report, except you write it in the source code)
* pass all the tests, that is, the following command executes without an error

In [ ]:
BEGIN SOLUTION
END SOLUTION
cd ~/notebooks/pl09_minc/test
make -B

* by default, the entire test stops as soon as any test fails. you may want to try `-k` option for make, to skip files that fails and go ahead to others

In [ ]:
BEGIN SOLUTION
END SOLUTION
cd ~/notebooks/pl09_minc/test
make -B -k

* write the summary of tests that passed/failed, in the online Excel `teamXX/pl09_minc/results.xlsx`
 * P : passed
 * C : cannot assemble (your compiler fails to produce assembly)
 * I : invalid assembly (your compiler produced an assembly code but it does not compile with gcc)
 * R : runtime error (your compiler produced an executable, but it does not terminate successfully)
 * W : wrong result (your compiler produced an executable that terminates successfully, but the output result is different from that of gcc-generated executable)

* members of each team must get togeter at least once to show their test results each other
* you are encouraged to get together in other occassions to discuss progress and help each other


* submit your work through Jupyter (modify and add files in place under ~/notebooks/pl09_minc); make sure you execute the above cell (`make -B -k`)
* submit "assignment-3 option 0" through UTOL
* all the essential work is submitted through Juptyer and Excel; you only have to submit a brief text that says "I submitted pl09_minc to Jupyter" in the ITC-LMS

## 6-2. Extra points [Level 2+]
* you get extra points by doing more than required above

* you can extend the minC language in any ways, but possible extensions include (but are not limited to):
  * syntactic extensions
    * [difficulty level 2] for loop
    * [difficulty level 2.5] initializing declaration (e.g., int x = y + 2)
  * type extensions and type checks
    * note that supporting a type other than long requires the compiler know the type of an expression; it's a heavy lifting
    * [difficulty level 3] pointers (long*, long**, etc.)
    * [difficulty level 4] floating point numbers (double)
    * [difficulty level 5] types of different sizes (int, float)
    * [difficulty level 6] structures and typedefs
  * optimization
    * use registers more aggressively
    * inline-expand function calls

* if you have done extra work beyond requirements, describe what you did in the extra.docx. clearly indicate the author of each section (who did what)